In [ ]:
import pathlib
import itertools

import pandas as pd
from IPython.display import display
import yaml

import dlsproc.assemble
import dlsproc.core

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

# Parameters

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/Sync/git/dlsproc/data


Directory in which the data is to be saved

In [ ]:
output_directory = data_directory / 'agregados'
assert output_directory.exists()
print(output_directory)

/home/manu/Sync/git/dlsproc/data/agregados


The name of the file **without** extension

In [ ]:
output_file = output_directory / '2018-2021'
print(f'{output_file} (existing? {output_file.exists()})')

/home/manu/Sync/git/dlsproc/data/agregados/2018-2021 (existing? False)


From the above *basename*

In [ ]:
pickle_file = output_file.with_suffix('.pickle')
parquet_file = output_file.with_suffix('.parquet')

# Processing

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

The directory for *outsiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/Sync/git/dlsproc/data/agregados


We infer whether we are working on a sample or the full dataset (useful later on)

In [ ]:
working_on_a_sample = outsiders_directory.parts[-2] == 'samples'
working_on_a_sample

False

The list of files to be processed. They were downloaded [here](https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesAgregacion.aspx). One could use something like
```
wget https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2018.zip
```

In [ ]:
zip_files = [
    'PlataformasAgregadasSinMenores_2018.zip',
    'PlataformasAgregadasSinMenores_2019.zip',
    'PlataformasAgregadasSinMenores_2020.zip',
    'PlataformasAgregadasSinMenores_2021.zip'
]
zip_files = [outsiders_directory/e for e in zip_files]
zip_files

[PosixPath('/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2018.zip'),
 PosixPath('/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2019.zip'),
 PosixPath('/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2020.zip'),
 PosixPath('/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2021.zip')]

In [ ]:
%%time
df = dlsproc.core.read_zips([outsiders_directory / e for e in zip_files])

Processing "/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2018.zip"
Processing "/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2019.zip"
Processing "/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2020.zip"
Processing "/home/manu/Sync/git/dlsproc/data/agregados/PlataformasAgregadasSinMenores_2021.zip"
CPU times: user 17min 54s, sys: 3.24 s, total: 17min 57s
Wall time: 17min 57s


Saving to *pickle*

In [ ]:
if not working_on_a_sample:
    df.to_pickle(pickle_file)

Saving to *parquet*

In [ ]:
parquet_df = dlsproc.assemble.parquet_amenable(df)
parquet_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                   452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                   452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                   452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                   

In [ ]:
if not working_on_a_sample:
    parquet_df.to_parquet(parquet_file)

## Making a sample

If we are working with sample files, a *parquet* sample file is created

In [ ]:
if working_on_a_sample:
    sample_df  = parquet_df.iloc[itertools.chain(range(10), range(-10, 0))]
    sample_df.to_parquet(outsiders_directory.parent / '2018-2021_20samples.parquet')